In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

In [2]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        
        # input is bs x 1 x h x w (h, w = 28, 28)
        self.conv1 = nn.Conv2d(1, 32, 5, padding=2)
        # output is bs x 32 x 28 x 28
        # after maxpool, bs x 32 x 14 x 14
        self.conv2 = nn.Conv2d(32, 64, 5, padding=2)
        # output is bs x 64 x 14 x 14
        # after maxpool, bs x 64 x 7 x 7
        self.fc1 = nn.Linear(64*7*7, 1024)
        self.fc2 = nn.Linear(1024, 10)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        # see in __init, the ouptut here is bs x 64 x 7 x 7
        x = x.view(-1, 64*7*7)
        x = F.relu(self.fc1(x))
        x = F.dropout(x)
        x = self.fc2(x)
        return F.log_softmax(x)

In [3]:
model = ConvNet()

In [6]:
bs = 50
train_loader = torch.utils.data.DataLoader(
datasets.MNIST('data', train=True, download=True, transform=transforms.ToTensor()),
              batch_size=bs, shuffle=True)

Processing...
Done!


In [7]:
test_loader = torch.utils.data.DataLoader(
datasets.MNIST('data', train=False, transform=transforms.ToTensor()),
batch_size= 1000)

In [9]:
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [ ]:
model.train()
train_loss = []
train_accu = []
i = 0
for epoch in range(15):
    for data, target in train_loader:
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()    # calc gradients
        train_loss.append(loss.data[0])
        optimizer.step()   # update gradients
        prediction = output.data.max(1)[1]   # first column has actual prob.
        accuracy = prediction.eq(target.data).sum()/bs*100
        train_accu.append(accuracy)
        if i % 1000 == 0:
            print('Train Step: {}\tLoss: {:.3f}\tAccuracy: {:.3f}'.format(i, loss.data[0], accuracy))
        i += 1

Train Step: 0	Loss: 2.303	Accuracy: 4.000
Train Step: 1000	Loss: 0.211	Accuracy: 94.000
Train Step: 2000	Loss: 0.140	Accuracy: 94.000
